# Welcome to your first assignment!

Instructions are below. Please give this a try, and look in the solutions folder if you get stuck (or feel free to ask me!)

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">Just before we get to the assignment --</h2>
            <span style="color:#f71;">I thought I'd take a second to point you at this page of useful resources for the course. This includes links to all the slides.<br/>
            <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">https://edwarddonner.com/2024/11/13/llm-engineering-resources/</a><br/>
            Please keep this bookmarked, and I'll continue to add more useful links there over time.
            </span>
        </td>
    </tr>
</table>

# HOMEWORK EXERCISE ASSIGNMENT

Upgrade the day 1 project to summarize a webpage to use an Open Source model running locally via Ollama rather than OpenAI

You'll be able to use this technique for all subsequent projects if you'd prefer not to use paid APIs.

**Benefits:**
1. No API charges - open-source
2. Data doesn't leave your box

**Disadvantages:**
1. Significantly less power than Frontier Model

## Recap on installation of Ollama

Simply visit [ollama.com](https://ollama.com) and install!

Once complete, the ollama server should already be running locally.  
If you visit:  
[http://localhost:11434/](http://localhost:11434/)

You should see the message `Ollama is running`.  

If not, bring up a new Terminal (Mac) or Powershell (Windows) and enter `ollama serve`  
And in another Terminal (Mac) or Powershell (Windows), enter `ollama pull llama3.2`  
Then try [http://localhost:11434/](http://localhost:11434/) again.

If Ollama is slow on your machine, try using `llama3.2:1b` as an alternative. Run `ollama pull llama3.2:1b` from a Terminal or Powershell, and change the code below from `MODEL = "llama3.2"` to `MODEL = "llama3.2:1b"`

In [2]:
# imports

import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display

In [3]:
# Constants

OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"

In [4]:
# Create a messages list using the same format that we used for OpenAI

messages = [
    {"role": "user", "content": "Describe some of the business applications of Generative AI"}
]

In [5]:
payload = {
        "model": MODEL,
        "messages": messages,
        "stream": False
    }

In [6]:
response = requests.post(OLLAMA_API, json=payload, headers=HEADERS)
print(response.json()['message']['content'])

Generative AI has numerous business applications across various industries. Here are some examples:

1. **Content Generation**: Generative AI can create high-quality content such as articles, social media posts, and product descriptions. This can help businesses reduce content creation costs and improve consistency.
2. **Image and Video Editing**: Generative AI-powered tools can edit images and videos to create realistic effects, remove blemishes, or even generate new images from existing ones.
3. **Customer Service Chatbots**: Generative AI can be used to create chatbots that can respond to customer inquiries and provide personalized support.
4. **Product Design**: Generative AI can assist in designing products by generating 2D and 3D models, providing suggestions for product development, and even creating prototypes.
5. **Marketing Campaigns**: Generative AI can help businesses create targeted marketing campaigns by analyzing customer data and behavior to generate personalized messag

# Introducing the ollama package

And now we'll do the same thing, but using the elegant ollama python package instead of a direct HTTP call.

Under the hood, it's making the same call as above to the ollama server running at localhost:11434

In [7]:
import ollama

response = ollama.chat(model=MODEL, messages=messages)
print(response['message']['content'])

Generative AI has numerous business applications across various industries, including:

1. **Content Creation**: Generative AI can generate high-quality content such as articles, social media posts, product descriptions, and more, saving time and resources for content creators.
2. **Marketing Automation**: AI-powered generative tools can create personalized marketing messages, emails, and ad copy, helping businesses personalize their marketing efforts and improve ROI.
3. **Customer Service Chatbots**: Generative AI enables the creation of highly realistic chatbots that can engage with customers, answer questions, and provide support, reducing the need for human customer service agents.
4. **Product Design and Development**: AI-powered generative tools can design new products, such as furniture, buildings, or electronic devices, based on user input, trends, and market demand.
5. **Finance and Accounting**: Generative AI can automate tasks such as financial forecasting, tax preparation, 

# NOW the exercise for you

Take the code from day1 and incorporate it here, to build a website summarizer that uses Llama 3.2 running locally instead of OpenAI

In [14]:
system_prompt = """You are a process engineer with a robotics background trying to improve the process and 
movement of the machine by gathering process sensor data and evaluating it into markdown."""
user_prompt = """
    i am encountering critical fft peaks at around 540 Hz and a process speed of 5600RPM. the process seems to cause issues on the resulting part,
    what parts should i check regarding damages and how should i go about solving my issue. try to compose a mathematical or analytical solution by 
    completing order analysis and potential part groups fitting that order
"""

def llm_dict_from(system_prompt, user_prompt):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]

def summarize():
    response = ollama.chat(
        model = 'llama3.2-vision',
        messages = llm_dict_from(system_prompt, user_prompt)
    )
    return response['message']['content']

def display_process_analysis():
    summary = summarize()
    display(Markdown(summary))

In [ ]:
display_process_analysis()

In [20]:
from anthropic import Anthropic
import markdown2
import os
import requests
from dotenv import load_dotenv

# Initialize the Anthropic client (you'll need to set your API key)
load_dotenv()
client = Anthropic(api_key=os.getenv('CLAUDE_API_KEY'))

# Create the system and user prompts
system_prompt = """You are a process engineer with a robotics background trying to improve the process and
movement of the machine by gathering process sensor data and evaluating it into markdown."""

user_prompt = """
    i am encountering critical fft peaks at around 540 Hz and a process speed of 5600RPM. the process seems to cause issues on the resulting part,
    what parts should i check regarding damages and how should i go about solving my issue. try to compose a mathematical or analytical solution by
    completing order analysis and potential part groups fitting that order
"""

def create_claude_messages(system_prompt, user_prompt):
    """
    Prepare messages for Claude API in the required format.
    
    Args:
        system_prompt (str): The system context for the analysis
        user_prompt (str): The specific user query or problem description
    
    Returns:
        list: A list of message dictionaries formatted for the Claude API
    """
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]

def analyze_process():
    """
    Send the prompts to Claude and retrieve the analysis.
    
    Returns:
        str: The generated markdown-formatted analysis response
    """
    try:
        # Create the messages
        messages = create_claude_messages(system_prompt, user_prompt)
        
        # Call the Claude API
        response = client.messages.create(
            model="claude-3-opus-20240229",  # You can choose a different model if preferred
            max_tokens=1000,
            messages=messages
        )
        
        # Extract the content from the response
        return response.content[0].text
    
    except Exception as e:
        return f"An error occurred: {str(e)}"

def display_process_analysis():
    """
    Analyze the process and display the result as formatted markdown.
    """
    # Get the analysis
    summary = analyze_process()
    
    # Convert to HTML (optional, depends on your display method)
    html_summary = markdown2.markdown(summary)
    
    # Display method depends on your environment 
    # For Jupyter:
    # from IPython.display import display, HTML
    # display(HTML(html_summary))
    
    # For standard output:
    print(summary)

# Run the analysis
if __name__ == "__main__":
    display_process_analysis()

An error occurred: Error code: 400 - {'type': 'error', 'error': {'type': 'invalid_request_error', 'message': 'Your credit balance is too low to access the Anthropic API. Please go to Plans & Billing to upgrade or purchase credits.'}}
